In [32]:
import pandas as pd
import numpy as np
from scipy import stats

bb_contract = pd.read_excel("../../Datasets/study-1-data.xlsx", sheet_name="BB contract")

pd.set_option('display.max_rows', 100)

In [33]:
bb_contract = bb_contract.drop("Unnamed: 6", axis=1)

In [34]:
bb_contract

,Experiment,Period,Player ID,Demand,Wholesale p.,Buyback p.,Player ID.1,Stock
0,1,1,Manufacturer 1,122,8,4,Retailer 1,100
1,1,2,Manufacturer 1,144,9,5,Retailer 1,105
2,1,3,Manufacturer 1,57,9,5,Retailer 1,139
3,1,4,Manufacturer 1,149,8,4,Retailer 1,102
4,1,5,Manufacturer 1,135,9,5,Retailer 1,135
...,...,...,...,...,...,...,...,...
875,3,36,Manufacturer 22,136,9,3,Retailer 22,80
876,3,37,Manufacturer 22,116,9,2,Retailer 22,60
877,3,38,Manufacturer 22,105,9,2,Retailer 22,65
878,3,39,Manufacturer 22,80,8,3,Retailer 22,105


In [35]:
bb_contract.columns

Index(['Experiment', 'Period', 'Player ID', 'Demand', 'Wholesale p.',
       'Buyback p.', 'Player ID.1', 'Stock'],
      dtype='object')

In [36]:
bb_contract

,Experiment,Period,Player ID,Demand,Wholesale p.,Buyback p.,Player ID.1,Stock
0,1,1,Manufacturer 1,122,8,4,Retailer 1,100
1,1,2,Manufacturer 1,144,9,5,Retailer 1,105
2,1,3,Manufacturer 1,57,9,5,Retailer 1,139
3,1,4,Manufacturer 1,149,8,4,Retailer 1,102
4,1,5,Manufacturer 1,135,9,5,Retailer 1,135
...,...,...,...,...,...,...,...,...
875,3,36,Manufacturer 22,136,9,3,Retailer 22,80
876,3,37,Manufacturer 22,116,9,2,Retailer 22,60
877,3,38,Manufacturer 22,105,9,2,Retailer 22,65
878,3,39,Manufacturer 22,80,8,3,Retailer 22,105


In [39]:
bb_contract['Sales'] = bb_contract[['Demand', 'Stock']].min(axis=1)
bb_contract['Leftovers'] = bb_contract['Stock'] - bb_contract['Sales']
bb_contract['Unmet_Demand'] = bb_contract['Demand'] - bb_contract['Sales']


bb_contract['Realized_Mfg_Profit'] = (bb_contract['Stock'] * bb_contract['Wholesale p.']) - (3 * bb_contract['Stock']) - bb_contract["Buyback p."] * bb_contract["Leftovers"]
bb_contract['Realized Retailer Profit'] = bb_contract['Sales'] * 12 - (bb_contract['Stock'] * bb_contract['Wholesale p.']) + bb_contract["Buyback p."] * bb_contract["Leftovers"] 

bb_contract['Realized_Mfg_Profit_Share'] = bb_contract['Realized_Mfg_Profit'] / (bb_contract['Realized_Mfg_Profit'] + bb_contract['Realized_Retailer_Profit'])

#wsp_contract['Optimal_Stock'] = 100*((12-wsp_contract['Wholesale p.'])/12 - 'Buyback p.') + 50
bb_contract['Optimal_Stock'] =  100 * (      (12 - bb_contract['Wholesale p.'] ) /  (   12 - bb_contract['Buyback p.']    )    )       +    50
bb_contract['Optimal_Stock'] = bb_contract['Optimal_Stock'].round()

bb_contract['Under/Overstock_wrt_Optimal'] = bb_contract['Stock'] - bb_contract['Optimal_Stock']
potential_demand = np.arange(51, 151)
expected_sales = []

for index, row in bb_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Stock'])

        cumulative_expected_sales += sales
        
        #if sales == row['Stock']:
         #   cumulative_expected_sales += (150 - demand_realization) * row['Stock']
          #  break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

bb_contract['Expected Sales'] = np.array(expected_sales)
bb_contract["Expected_Leftovers"] = bb_contract["Optimal_Stock"] - bb_contract["Expected Sales"]

#wsp_contract['Expected Retailer Profit'] = (wsp_contract['Expected Sales']  12) - (wsp_contract['Wholesale p.'] * wsp_contract['Stock']) + wsp_contract["Expected_Leftovers"] * ..buyback price..
bb_contract['Expected Retailer Profit'] = (bb_contract['Expected Sales'] * 12) - (bb_contract['Wholesale p.'] * bb_contract['Stock']) + bb_contract["Expected_Leftovers"] * bb_contract["Buyback p."]

#wsp_contract['Expected Mfg Profit'] = (wsp_contract['Stock']  wsp_contract['Wholesale p.']) - (3  wsp_contract['Stock']) - ..buybackprice.. * wsp_contract["Expected_Leftovers"]
bb_contract['Expected Mfg Profit'] = (bb_contract['Stock'] * bb_contract['Wholesale p.']) - (3 * bb_contract['Stock']) - bb_contract["Buyback p."] * bb_contract["Expected_Leftovers"]

bb_contract['Expected Mfg. Profit Share'] = bb_contract['Expected Mfg Profit'] / (bb_contract['Expected Mfg Profit'] + bb_contract['Expected Retailer Profit']) #bb_contract.apply(lambda row: row['Expected Mfg Profit'] / row['Total Expected Revenue'] if row['Total Expected Revenue'] != 0 else 0, axis=1)
potential_demand = np.arange(51, 151)
expected_sales = []

for index, row in bb_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Optimal_Stock'])

        cumulative_expected_sales += sales
        
        #if sales == row['Stock']:
         #   cumulative_expected_sales += (150 - demand_realization) * row['Stock']
          #  break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

bb_contract['Predicted Sales'] = np.array(expected_sales)
bb_contract['Predicted Leftovers'] = bb_contract['Optimal_Stock'] - bb_contract['Predicted Sales'] 

bb_contract['Predicted Retailer Profit'] = bb_contract['Predicted Sales'] * 12 - (bb_contract['Wholesale p.'] * bb_contract['Optimal_Stock']) + bb_contract['Predicted Leftovers'] * bb_contract["Buyback p."]

bb_contract['Predicted Mfg Profit'] = (bb_contract['Optimal_Stock'] * bb_contract['Wholesale p.']) - (3 * bb_contract['Optimal_Stock']) - bb_contract["Buyback p."] * bb_contract['Predicted Leftovers']

bb_contract['Predicted Mfg. Profit Share'] = bb_contract['Predicted Mfg Profit'] / (bb_contract['Predicted Mfg Profit'] + bb_contract['Predicted Retailer Profit'])

In [40]:
bb_contract.to_csv('./curr_data_bb.csv', index = False)